# Salvar os arquivos iniciais

In [8]:
import pandas as pd
import requests
import pathlib

Salvando os arquivos já baixados em .csv em variáveis

In [ ]:
df_nibrs_offenses = pd.read_csv(r"data\Crime_Data_from_2020_to_Present.csv")
df_nibrs_victoms = pd.read_csv(r"data\LAPD_NIBRS_Victims_Dataset.csv")
df_crime_data = pd.read_csv(r"data\Crime_Data_from_2020_to_Present.csv")

## Extraindo dados da API da NOAA
Como o dataset está disponível apenas através da API disponibilizada pela NOAA *(National Oceanic and Atmospheric Administration)*, foi preciso registrar um **token** no site da agência, que foi enviado por email

In [42]:
#config
token = "WZzNRktcksOxuQRDpDgdTyqlNgVkqtBd"
endpoint = "https://www.ncei.noaa.gov/access/services/data/v1"

params = {
    "dataset": "daily-summaries",
    "stations": "USW00023174",
    "startDate": "2024-03-07",
    "endDate": "2025-10-31",
    "format": "csv",
    "units": "metric"
}

#request
response = requests.get(endpoint, params=params, headers={"token": token})


Salvar o arquivo extraído da API em um .csv na pasta */data*

In [ ]:
from pathlib import Path

root = Path.cwd() #sobe um nível (.parent) pra chegar na raiz v 
data_dir = root/ "data" #aponta pra pasta data/

save_csv = data_dir / "LA-wheather-daily-2024-2025.csv" #salva na pasta data/ --> agr é só usar essa função pra salvar qqlr arquivo na pasta 

#salva o arquivo extraído da API na pasta
with open (save_csv, "wb") as f:
    f.write(response.content)
    
print("Arquivo salvo em: ", save_csv)

df_global_hourly = pd.read_csv(r"data\LA-wheather-daily-2024-2025.csv")